# Лекция 8. Классы (продолжение)

* Атрибуты
* Наследование

# Атрибуты

Важно различать, что такое атрибуты класса и атрибуты экземпляра

In [64]:
class MySecondClass:
    # это атрибут класса
    var1 = ["test"]
    
    # это метод класса
    def Method1(self):
        pass
    
    def __init__(self):
        # А вот это создаст атрибут экземпляра во время его создания
        self.var2 = ["test2"]
    
a = MySecondClass()

In [65]:
print(a.var1)
print(a.var2)

['test']
['test2']


In [66]:
a.__dict__

{'var2': ['test2']}

In [67]:
a.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'var1': ['test'],
              'Method1': <function __main__.MySecondClass.Method1(self)>,
              '__init__': <function __main__.MySecondClass.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'MySecondClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MySecondClass' objects>,
              '__doc__': None})

In [68]:
# Создадим второй экземпляр

b = MySecondClass()

print(f'{a.var1=}')
print(f'{a.var2=}')
print(f'{b.var1=}')
print(f'{b.var2=}')

a.var1=['test']
a.var2=['test2']
b.var1=['test']
b.var2=['test2']


In [6]:
# Немного изменим второй экземпляр

# изменим атрибут экземпляра класса
b.var2.append("!")

print(f'{a.var1=}')
print(f'{a.var2=}')
print(f'{b.var1=}')
print(f'{b.var2=}')

a.var1=['test']
a.var2=['test2']
b.var1=['test']
b.var2=['test2', '!']


In [7]:
# Немного изменим второй экземпляр

# изменим атрибут класса
b.var1.append("!")

print(f'{a.var1=}')
print(f'{a.var2=}')
print(f'{b.var1=}')
print(f'{b.var2=}')

a.var1=['test', '!']
a.var2=['test2']
b.var1=['test', '!']
b.var2=['test2', '!']


In [8]:
# Немного изменим второй экземпляр

# изменим атрибут класса
b.var1 = 1

# не совсем то, что ожидается
print(f'{a.var1=}')
print(f'{a.var2=}')
print(f'{b.var1=}')
print(f'{b.var2=}')

a.var1=['test', '!']
a.var2=['test2']
b.var1=1
b.var2=['test2', '!']


При присваивании значения, будет создан атрибут в экземпляре, если его нет, что приводит к сокрытию атрибута класса.

In [9]:
print(f'{b.var1=}')
print(f'{b.__class__.var1=}')

b.__dict__

b.var1=1
b.__class__.var1=['test', '!']


{'var2': ['test2', '!'], 'var1': 1}

# Наследование

Наследование является очень мощным механизмом, который позволяет повторно эффективно использовать существующий код. В Python'е через наследование можно настраивать поведение классов. __Помним, что self - это ссылка на текущий экземпляр класса.__

In [73]:
class C2:
    A = "hi"
    pass

class C3:
    B = "ho"
    pass

# Наследование
# Мы просто перечисляем классы внутри скобок
class C1(C2, C3):
    pass

In [74]:
a = C1()
a.A, a.B

('hi', 'ho')

В Python наследование, фактически, позволяет выстроить цепочку объектов, среди которых будет искать необходимый атрибут.

> `isinstance(obj, class_or_tuple)` - позволяет проверить, является ли объект потомком какого-то класса

In [11]:
a = C1()
print(isinstance(a, C2))

True


In [12]:
a = C2()
print(isinstance(a, C3))

False


In [13]:
# Это можно делать для любых объектов
print(isinstance(1, str))
print(isinstance(1, int))

False
True


## Перегрузка методов 

In [14]:
class Employee:
    def __init__(self):
        pass
    
    def coefficient(self):
        return 1.0
    
    def computeSalary(self):
        return 10000 * self.coefficient()
    
    def promote(self):
        pass
    
    def retire(self):
        pass

Если мы захочем ввести новый тип сотрудников со специальными правками, то мы могли бы просто скопировать-вставить этот класс и в этой копии что-то изменить. Это неправильный путь. Лучше использовать наследование:

In [75]:
class Engineer(Employee):
    def computeSalary(self):
        return 20000 * self.coefficient()

In [16]:
person1 = Employee()
person2 = Employee()
person3 = Engineer()

for person in [person1, person2, person3]:
    print(person.computeSalary())

10000.0
10000.0
20000.0


## Особенности перегрузки

Позволяют настроить внутреннее состояние экземпляра класса

In [79]:
class Employee:
    def __init__(self, name, pay, job="Employee"):
        self._name = name
        self._pay = pay
        self._job = job
        
    def giveRaise(self, percent):
        self._pay *= (1 + percent)
    
    def computeSalary(self):
        return self._pay
    
    def __str__(self):
        return f'[{self._job}] {self._name}: {self.computeSalary()}'

In [80]:
person1 = Employee("John", 10000)
person1.giveRaise(.10)

print(person1)

[Employee] John: 11000.0


Нам нужно модифицировать метод, отвечающий за повышение зарплаты

In [81]:
class Engineer(Employee):
    def giveRaise(self, percent, bonus=0.1):
        # Плохой способ, так как копирование-вставка
        self._pay *= (1 + percent + bonus)

In [82]:
class Engineer(Employee):
    def giveRaise(self, percent, bonus=0.1):
        # Оптимальный, мы по полной используем старый код
        #Employee.giveRaise(self, percent + bonus)  
        super().giveRaise(percent + bonus)

In [83]:
# обратите внимание, что вызван конструктор предка
person2 = Engineer("Jack", 10000)
person2.giveRaise(.00)

print(person2)

[Employee] Jack: 11000.0


## Перегрузка конструкторов

Теперь нам хотелось бы настроить атрибут `_job`, так как его значение логически зависит от конкретного класса

In [90]:
class Engineer(Employee):
    def __init__(self, name, pay):
        #self._pay = pay
        #self._name = name
        self._job = "Engineer"
        
    def giveRaise(self, percent, bonus=0.1):
        Employee.giveRaise(self, percent + bonus)  
     
# получаем ошибку, так как тут у объекта уже есть конструктор
person2 = Engineer("Jack", 10000)
person2.giveRaise(.1)

AttributeError: 'Engineer' object has no attribute '_pay'

In [23]:
# а так у нас будут отсутствовать атрибуты из Employee

person2 = Engineer()
person._name

AttributeError: 'Engineer' object has no attribute '_name'

In [91]:
class Engineer(Employee):
    def __init__(self, name, pay):
        # нам нужно явно проинициализировать Employee
        super().__init__(name, pay, job="Engineer")
        # Employee.__init__(self, name, pay, job="Engineer")
        
        
    def giveRaise(self, percent, bonus=0.1):
        Employee.giveRaise(self, percent + bonus)  

In [93]:
person2 = Engineer("Jack", 10000)
print(person2)

[Engineer] Jack: 10000


In [94]:
person2.__dict__

{'_name': 'Jack', '_pay': 10000, '_job': 'Engineer'}

# super

> super([type[, object]]) - данная функция ищет метод в родительском классе, игнорируя тип текущего объекта



In [95]:
class Engineer(Employee):
    def __init__(self, name, pay):
        # нам нужно явно проинициализировать Employee
        super().__init__(name, pay, job="Engineer")
        
    def giveRaise(self, percent, bonus=0.1):
        super().giveRaise(percent + bonus)  

In [96]:
person2 = Engineer("Jack", 10000)
print(person2)

[Engineer] Jack: 10000


## Ромбовидное наследование

Поиск метода осуществляется согласно [MRO](https://ru.wikipedia.org/wiki/C3-%D0%BB%D0%B8%D0%BD%D0%B5%D0%B0%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F).

In [97]:
# Метод будет выбран согласно MRO из классов-родителей

class C1:
    def print(self):
        print("C1")
        
class C2:
    def print(self):
        print("C2")
        
class C3(C1, C2):
    pass

# Берется первый в списке
C3().print()

C1


In [29]:
# Фактически тоже самое, но написано руками

class C1:
    def print(self):
        print("C1")
        
class C2:
    def print(self):
        print("C2")
        
class C3(C1, C2):
    def print(self):
        super().print()

# Берется первый в списке
C3().print()

C1


In [98]:
# Можно выбрать конкретную реализацию, если нет нужды полагаться на MRO

class C3(C1, C2):
    def print(self):
        C2.print(self)
        
C3().print()

C2


Есть случаи, когда `super()` более чем оправдан - это использование инициализации в случае сложной зависимости классов

In [102]:
class A1:
    def __init__(self):
        print("A1")

class B1(A1):
    def __init__(self):
        super().__init__()
        print("B1")
        
class B2(A1):
    def __init__(self):
        super().__init__()
        print("B2")
        
class C1(B1, B2):
    def __init__(self):
        super().__init__()
        print("C1")

In [103]:
C1()

A1
B2
B1
C1


# Экранировка имен переменных

Можно использовать автоматическую экранировку имен, чтобы не испортить поведение родительских классов

In [105]:
# Проблема

class C1:
    X = 15
    def methodC1(self):
        print(self.X)
        
class C2:
    X = 51
    def methodC2(self):
        print(self.X)
        
class C3(C1, C2):
    pass

a = C3()
# не то, что хотели бы получить
a.methodC1()
a.methodC2()

15
15


In [109]:
# Решение

class C1:
    __X = 15
    def methodC1(self):
        print(self.__X)
        
class C2:
    __X = 51
    def methodC2(self):
        print(self.__X)
        
class C3(C1, C2):
    pass

a = C3()

# то, что хотели бы получить
a.methodC1()
a.methodC2()

15
51


In [107]:
# Даст ошибку

a.__X

AttributeError: 'C3' object has no attribute '__X'

In [36]:
# Посмотрим, как это работает
dir(a)

['_C1__X',
 '_C2__X',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'methodC1',
 'methodC2']

In [108]:
a._C1__X

15

## Примеси

Достаточно интересный способ использования наследования. Можно создать класс, кототый реализует какое-то четкое поведение, но самостоятельно никогда не используется или не может быть использован. Нужен только для уточнения поведения других классов.

In [37]:
class AttributeInsiderMixin:
    """Примесь, которая выводит атрибуты экземпляра"""
    def insideJob(self):
        print(self.__class__.__name__, ":")
        for key in self.__dict__:
            print(f"\t => {key:>15s} = {self.__dict__[key]}")

In [38]:
AttributeInsiderMixin?

Init signature: AttributeInsiderMixin()
Docstring:      Примесь, которая выводит атрибуты экземпляра
Type:           type
Subclasses:     


In [110]:
class Engineer(Employee, AttributeInsiderMixin):
    def __init__(self, name, pay):
        # нам нужно явно проинициализировать Employee
        Employee.__init__(self, name, pay, job="Engineer")
        
        # использование двойного подчеркивания экранирует имя переменной
        self.__test = "testing"
        
    def giveRaise(self, percent, bonus=0.1):
        Employee.giveRaise(self, percent + bonus)  
        
    def print(self):
        print(self.__test)

In [111]:
person = Engineer("Jack", 10000)

person.insideJob()

Engineer :
	 =>           _name = Jack
	 =>            _pay = 10000
	 =>            _job = Engineer
	 => _Engineer__test = testing


Можно пойти дальше и сделать это чуть удобнее

In [41]:
class PrettyPrintMixin:
    def __str__(self):
        res = "%s: \n" % self.__class__.__name__
        for key in self.__dict__:
            res += f"\t => {key:>15s} = {self.__dict__[key]}\n"
        res += "\n"
        return res
        
    def __repr__(self):
        return self.__str__()

In [112]:
# Порядок наследования важен, так как мы уже определили 
# свой метод __str__ в Employee
#
class Engineer(PrettyPrintMixin, Employee):
    def __init__(self, name, pay):
        # нам нужно явно проинициализировать Employee
        Employee.__init__(self, name, pay, job="Engineer")
        
        # использование двойного подчеркивания экранирует имя переменной
        self.__test = "testing"
        
    def giveRaise(self, percent, bonus=0.1):
        Employee.giveRaise(self, percent + bonus)  

In [113]:
Engineer("Jack", 10000)

Engineer: 
	 =>           _name = Jack
	 =>            _pay = 10000
	 =>            _job = Engineer
	 => _Engineer__test = testing


## Функторы

Можно создать класс, которые ведет себя как функция

In [44]:
# Проблема

def line(x, k, b):
    return x*k + b

line(10, -2, 5)

-15

In [45]:
# Решение

class Line:
    def __init__(self, k, b):
        self.__k = k
        self.__b = b
        
    def __call__(self, x):
        return x*self.__k + self.__b
    
f = Line(-2, 5)
f(10)

-15

Можно использовать функторы в любых местах, где ожидается функция.

In [46]:
class Filter:
    def __init__(self, thr):
        self._thr = thr
        
    def __call__(self, x):
        return x < self._thr
    
functor = Filter(10)
functor(17)

# Создадим массив чисел
series = [i for i in range(20)]
print(series)


# проверим, можем ли мы использовать класс как функцию
print(list(map(functor, series)))
# эквивалент
print(list(map(lambda x: x < 10, series)))

print(list(filter(functor, series)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False]
[True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Абстрактные классы

Можно создать класс, который нельзя использовать, но он будет являться подсказкой для программистов, которые будут использовать ваш фреймворк.

In [117]:
class Model:
    def fit(self, X, y):
        raise NotImplementedError()
        
    def predict(self, X):
        raise NotImplementedError()
    
        
m = Model()
m.fit("", "")

NotImplementedError: 

In [116]:
class MyModel(Model):
    def fit(self, X, y):
        return True
        
    def predict(self, X):
        return True
    
m = MyModel()
m.fit("", "")

'No string'

# Перегрузка операций

С помощью классов можно настраивать поведение классов при использовании в специфических ситуациях.  Мы уже знакомы со __str__ и __repr__, но есть более интересные аспекты перегрузки операций

In [49]:
class MyClass:
    """Пример работы с длинной"""
    def __len__(self):        
        return 100
    
len(MyClass())

100

In [50]:
class MyClass:
    """Пример работы с преобразованием к логическому типу"""
    def __bool__(self):        
        return False
    
bool(MyClass())

False

In [51]:
class MyClass:
    """Пример работы с индексами"""
    def __init__(self):
        self._my_dict = {}
        
    def __setitem__(self, index, value):
        self._my_dict[index] = value
        
    def __getitem__(self, index):
        if index not in self._my_dict:
            # Мы создаем исключительную ситуацию
            raise IndexError                                
        else:
            return self._my_dict[index]
    
m = MyClass()
m[0] = "asd"
m[1] = "sdf"
m[2] = "fjh"
m[5] = "zxc"

m[5]

'zxc'

In [52]:
# Просто проходиться по индексам от 0 до пока не получит исключение IndexError
for el in m:
    print(el)

asd
sdf
fjh


In [53]:
# Создание итератора

class RangeSquare:
    """Пример работы с протоколом итераций"""
    def __init__(self, start, stop):
        self._start = start
        self._value = start - 1
        self._stop = stop

    def __iter__(self):
        """
        Должен возвращать объект-итератор (не обязательно сам класс)
        """
        return self
    
    def __next__(self):
        if self._value >= self._stop:
            # self._value = self._start - 1
            raise StopIteration
        self._value += 1
        return self._value**2
    
m = RangeSquare(2, 5)
it = iter(m)
next(it), next(it)

(4, 9)

In [54]:
m = RangeSquare(2, 5)
for v in m:
    print(v)

4
9
16
25


In [55]:
# А вот второй раз не сработает
for v in m:
    print(v)

и много других операций

# Почитать

* [Паттерны ООП](https://refactoring.guru/ru/design-patterns)
* [UML](https://www.tutorialspoint.com/uml/index.htm)

# Домашняя работа

## Задача

Создать абстрактный класс `Spline`, который является базовой заготовкой для [сплайнов](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%BB%D0%B0%D0%B9%D0%BD). Данный класс должен определять некий обязательный интерфейс, также данный класс должен быть функтором.

Создать два класса `LinearSpline`(обязательно) и `CubicSpline`(опционально), которые являются потомками `Spline` и реализуют его интерфейс. `LinearSpline` реализует интерполяцию полиномом первой степени, `QuadraticSpline` - полиномом третьей степени.

Таким образом, в результате вы должны получить сплайн-объект, который как-то инициализируется с помощью $N$ пар точек $(x, y)$ и позволяет получить значение $y$ для любого значения $x$. Фактически, ваш класс будет восстанавливать функцию $y = f(x)$ с некоторой точностью.

__Сторонние библиотеки и модули использовать нельзя__